In [10]:
import gpxpy
import folium

# Load the GPX file
gpx_file_path = 'HMHR2023-05KM-v2.5.gpx'  # Replace with your GPX file path

# Parse the GPX file
with open(gpx_file_path, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extract the problematic coordinates
coordinates = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            coordinates.append((point.latitude, point.longitude))

# Visualize the current route using folium
current_map = folium.Map(location=coordinates[0], zoom_start=14)
folium.PolyLine(coordinates, color='blue', weight=5).add_to(current_map)

# Render the current map to verify the problem
current_map.save('original_route.html')

print("Route map saved to 'original_route.html'. Open it to identify incorrect points.")

# Fix the problematic points
# Replace these with your corrections based on the problematic areas
fixed_coordinates = [
    # Replace with the corrected points
    (21.0278, 105.8342),  # Example start
    (21.0300, 105.8370),
    (21.0330, 105.8395),
    (21.0358, 105.8423)   # Example end
]

# Create a new GPX file with the corrected coordinates
new_gpx = gpxpy.gpx.GPX()
new_track = gpxpy.gpx.GPXTrack()
new_gpx.tracks.append(new_track)
new_segment = gpxpy.gpx.GPXTrackSegment()
new_track.segments.append(new_segment)

for lat, lon in fixed_coordinates:
    new_segment.points.append(gpxpy.gpx.GPXTrackPoint(lat, lon))

# Save the corrected GPX file
corrected_gpx_path = 'corrected_route.gpx'
with open(corrected_gpx_path, 'w') as corrected_file:
    corrected_file.write(new_gpx.to_xml())

# Visualize the corrected route
corrected_map = folium.Map(location=fixed_coordinates[0], zoom_start=14)
folium.PolyLine(fixed_coordinates, color='green', weight=5).add_to(corrected_map)

# Save the corrected route map
corrected_map.save('corrected_route.html')

print(f"Corrected GPX file saved to '{corrected_gpx_path}' and map saved to 'corrected_route.html'.")


Route map saved to 'original_route.html'. Open it to identify incorrect points.
Corrected GPX file saved to 'corrected_route.gpx' and map saved to 'corrected_route.html'.


In [7]:
import gpxpy
import gpxpy.gpx
from datetime import datetime, timedelta

# Path to your original GPX file
gpx_file_path = 'HMHR2023-05KM-v2.5.gpx'

# Custom start point (latitude, longitude, elevation, time)
custom_start_point = {
    "lat": 21.0278,  # Replace with your desired latitude
    "lon": 105.8342,  # Replace with your desired longitude
    "ele": 10.0,      # Replace with your desired elevation
    "time": datetime(2024, 12, 1, 7, 0, 0)  # Replace with your desired start time
}

# Custom end point (latitude, longitude, elevation)
custom_end_point = {
    "lat": 21.032287222829723,  # Replace with your desired latitude
    "lon": 105.85064269723448,  # Replace with your desired longitude
    "ele": 15.0       # Replace with your desired elevation
}

# Desired pace in seconds per kilometer
desired_pace_seconds_per_km = 300

# Read and parse the GPX file
with open(gpx_file_path, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Update the start point
if gpx.tracks and gpx.tracks[0].segments and gpx.tracks[0].segments[0].points:
    start_point = gpx.tracks[0].segments[0].points[0]
    start_point.latitude = custom_start_point["lat"]
    start_point.longitude = custom_start_point["lon"]
    start_point.elevation = custom_start_point["ele"]
    start_point.time = custom_start_point["time"]

# Initialize variables for timing
start_time = custom_start_point["time"]
total_distance = 0.0
previous_point = None

# Update the timing and distance for intermediate points
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            if previous_point:
                # Calculate distance from the previous point
                distance = point.distance_2d(previous_point)
                total_distance += distance

                # Calculate the time increment for the distance
                time_increment = timedelta(seconds=(distance / 1000) * desired_pace_seconds_per_km)
                start_time += time_increment

            # Set the updated time for the current point
            point.time = start_time

            previous_point = point

# Update the end point
if gpx.tracks and gpx.tracks[0].segments:
    end_segment = gpx.tracks[0].segments[0]
    end_point = end_segment.points[-1]
    end_point.latitude = custom_end_point["lat"]
    end_point.longitude = custom_end_point["lon"]
    end_point.elevation = custom_end_point["ele"]
    end_point.time = start_time  # Final time based on the last point

# Save the updated GPX file
updated_gpx_file_path = 'custom_start_end_gpx_file.gpx'
with open(updated_gpx_file_path, 'w') as updated_gpx_file:
    updated_gpx_file.write(gpx.to_xml())

print(f"Updated GPX file with custom start and end points saved to: {updated_gpx_file_path}")


Updated GPX file with custom start and end points saved to: custom_start_end_gpx_file.gpx


In [9]:
import folium
import gpxpy

# Path to your GPX file
gpx_file_path = 'custom_start_end_gpx_file.gpx'

# Read the GPX file
with open(gpx_file_path, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extract coordinates from the GPX file
coordinates = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            coordinates.append((point.latitude, point.longitude))

# Create a map centered around the first point
if coordinates:
    start_point = coordinates[0]
    gpx_map = folium.Map(location=start_point, zoom_start=14)

    # Add the GPX route as a polyline
    folium.PolyLine(coordinates, color='blue', weight=5, opacity=0.7).add_to(gpx_map)

    # Add markers for start and end points
    folium.Marker(location=coordinates[0], popup="Start Point", icon=folium.Icon(color='green')).add_to(gpx_map)
    folium.Marker(location=coordinates[-1], popup="End Point", icon=folium.Icon(color='red')).add_to(gpx_map)

    # Display the map in Jupyter or save to an HTML file
    gpx_map.save('gpx_route_map.html')
    print("Map saved as 'gpx_route_map.html'. Open it in a browser to view.")
else:
    print("No coordinates found in the GPX file.")


Map saved as 'gpx_route_map.html'. Open it in a browser to view.
